In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from env import user, host, password
import os

# Zillow Clustering

In [5]:
def get_connection(db, username=user, host=host, password=password):
    '''
    Creates a connection URL
    '''
    return f'mysql+pymysql://{username}:{password}@{host}/{db}'
    


### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.
- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.
- Only include properties that include a latitude and longitude value.

In [21]:
def new_zillow_data():
    '''
    Returns zillow into a dataframe
    '''
    sql_query = '''  SELECT *
    FROM properties_2017
    LEFT OUTER JOIN airconditioningtype 
    USING (airconditioningtypeid)
    LEFT OUTER JOIN architecturalstyletype
    USING (architecturalstyletypeid)
    LEFT OUTER JOIN buildingclasstype 
    USING (buildingclasstypeid)
    LEFT OUTER JOIN heatingorsystemtype
    USING (heatingorsystemtypeid)
    LEFT OUTER JOIN predictions_2017
    USING (id)
    INNER JOIN (
    SELECT id, MAX(transactiondate) as last_trans_date 
    FROM predictions_2017
    GROUP BY id
    ) predictions ON predictions.id = properties_2017.id AND predictions_2017.transactiondate = predictions.last_trans_date
    LEFT OUTER JOIN propertylandusetype
    USING(propertylandusetypeid)
    LEFT OUTER JOIN storytype
    ON storytype.storytypeid = properties_2017.storytypeid
    LEFT OUTER JOIN typeconstructiontype
    ON typeconstructiontype.typeconstructiontypeid = properties_2017.typeconstructiontypeid
    JOIN unique_properties
    ON unique_properties.parcelid = properties_2017.parcelid
    WHERE latitude IS NOT NULL and longitude IS NOT NULL; '''
    df = pd.read_sql(sql_query, get_connection('zillow'))
    return df 

def get_zillow_data():
    '''get connection, returns zillow into a dataframe and creates a csv for us'''
    if os.path.isfile('zillow.csv'):
        df = pd.read_csv('zillow.csv', index_col=0)
    else:
        df = new_zillow_data()
        df.to_csv('zillow.csv')
    return df

def nulls_by_col(df):
    '''Returns a dataframe that shows the amount of null values in each column'''
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = num_missing / rows * 100
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 'percent_rows_missing': prcnt_miss})
    return cols_missing

def nulls_by_row(df):
    '''Returns a dataframe that shows the amount of null values in each row'''
    num_missing = df.isnull().sum(axis=1)
    prcnt_miss = num_missing / df.shape[1] * 100
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'customer_id': 'num_rows'}).reset_index()
    return rows_missing

In [22]:
zillow_df = get_zillow_data()
zillow_df.shape

/opt/homebrew/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (61,62,71,73) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


(77614, 74)

In [23]:
zillow_df.parcelid.unique

<bound method Series.unique of 0        10754147
1        10759547
2        10843547
3        10859147
4        10879947
           ...   
77609    11212539
77610    11212639
77611    11212962
77612    11213162
77613    11213262
Name: parcelid, Length: 77614, dtype: int64>

### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [24]:
zillow_df.shape

(77614, 74)

In [25]:
zillow_df.describe().T

,count,mean,std,min,25%,50%,75%,max
propertylandusetypeid,77614.0,2.599353e+02,1.696516e+01,3.100000e+01,2.610000e+02,2.610000e+02,2.610000e+02,2.750000e+02
id,77614.0,3.880650e+04,2.240538e+04,0.000000e+00,1.940325e+04,3.880650e+04,5.820975e+04,7.761300e+04
heatingorsystemtypeid,48475.0,4.043012e+00,3.231120e+00,1.000000e+00,2.000000e+00,2.000000e+00,7.000000e+00,2.400000e+01
buildingclasstypeid,383.0,3.775457e+00,4.595960e-01,1.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,5.000000e+00
architecturalstyletypeid,151.0,7.430464e+00,2.629347e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
...,...,...,...,...,...,...,...,...
logerror,77614.0,1.675218e-02,1.708879e-01,-4.655420e+00,-2.432860e-02,6.669808e-03,3.929424e-02,5.262999e+00
id.1,77614.0,3.880650e+04,2.240538e+04,0.000000e+00,1.940325e+04,3.880650e+04,5.820975e+04,7.761300e+04
storytypeid.1,45.0,7.000000e+00,0.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00
typeconstructiontypeid.1,166.0,5.987952e+00,1.552301e-01,4.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00,6.000000e+00


In [26]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77614 entries, 0 to 77613
Data columns (total 74 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         77614 non-null  float64
 1   id                            77614 non-null  int64  
 2   heatingorsystemtypeid         48475 non-null  float64
 3   buildingclasstypeid           383 non-null    float64
 4   architecturalstyletypeid      151 non-null    float64
 5   airconditioningtypeid         21457 non-null  float64
 6   parcelid                      77614 non-null  int64  
 7   basementsqft                  45 non-null     float64
 8   bathroomcnt                   77614 non-null  float64
 9   bedroomcnt                    77614 non-null  float64
 10  buildingqualitytypeid         50322 non-null  float64
 11  calculatedbathnbr             74391 non-null  float64
 12  decktypeid                    513 non-null    float64
 13  f

### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.


In [27]:
def missing_values_table(df):
    '''
    this function takes a dataframe as input and will output metrics for missing values, and the percent of that column that has missing values
    '''
        # Total missing values
    mis_val = df.isnull().sum()
        # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
           "columns that have missing values.")
        # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [28]:
missing_values_table(zillow_df)

Your selected dataframe has 74 columns.
There are 56columns that have missing values.


,Missing Values,% of Total Values
basementsqft,77569,99.9
storydesc,77569,99.9
storytypeid,77569,99.9
storytypeid.1,77569,99.9
yardbuildingsqft26,77548,99.9
fireplaceflag,77489,99.8
architecturalstyletypeid,77463,99.8
architecturalstyledesc,77463,99.8
typeconstructiondesc,77448,99.8
typeconstructiontypeid,77448,99.8


### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [29]:
zillow = pd.DataFrame(zillow_df.isnull().sum(axis =1), columns = ['num_cols_missing']).reset_index().groupby('num_cols_missing').count().reset_index().rename(columns = {'index': 'num_rows' })
zillow

,num_cols_missing,num_rows
0,25,12
1,26,21
2,27,42
3,28,43
4,29,160
5,30,351
6,31,4281
7,32,2851
8,33,8974
9,34,11544


In [30]:
zillow['%_cols_missing'] = zillow.num_cols_missing/zillow.shape[1]

In [31]:
def cols_missing(df):
    df = pd.DataFrame(df.isnull().sum(axis =1), columns = ['num_cols_missing'])\
    .reset_index().groupby('num_cols_missing').count()\
    .reset_index().rename(columns = {'index': 'num_rows' })
    df['pct_cols_missing'] = df.num_cols_missing/df.shape[1] 
    return df

In [32]:
cols_missing(zillow_df)

,num_cols_missing,num_rows,pct_cols_missing
0,25,12,12.5
1,26,21,13.0
2,27,42,13.5
3,28,43,14.0
4,29,160,14.5
5,30,351,15.0
6,31,4281,15.5
7,32,2851,16.0
8,33,8974,16.5
9,34,11544,17.0


Currently, I have no clue on how I want to approach missing values. Perhaps I could Impute them, or I could drop them entirely. I'd like to avoid dropping because I only have 77000 rows and I don't want to prune down my dataset too much. It will already be reduced from dropping outliers and also during the train test split.

## Acquire

### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [33]:
zillow_df.shape

(77614, 74)

In [34]:
single_use_codes = [261, 262, 263, 266, 268, 273, 276, 279]
zillow_df = zillow_df[zillow_df['propertylandusetypeid'].isin(single_use_codes)]

zillow_df

,propertylandusetypeid,id,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,basementsqft,bathroomcnt,bedroomcnt,...,logerror,transactiondate,id.1,last_trans_date,propertylandusedesc,storytypeid.1,storydesc,typeconstructiontypeid.1,typeconstructiondesc,parcelid.2
1,261.0,1,NaN,NaN,NaN,NaN,10759547,NaN,0.0,0.0,...,0.055619,2017-01-01,1,2017-01-01,Single Family Residential,NaN,NaN,NaN,NaN,10759547
14,266.0,14,NaN,NaN,NaN,NaN,11142747,NaN,0.0,0.0,...,-0.008935,2017-01-02,14,2017-01-02,Condominium,NaN,NaN,NaN,NaN,11142747
15,261.0,15,NaN,NaN,NaN,NaN,11193347,NaN,0.0,0.0,...,0.008669,2017-01-02,15,2017-01-02,Single Family Residential,NaN,NaN,NaN,NaN,11193347
16,261.0,16,NaN,NaN,NaN,NaN,11215747,NaN,0.0,0.0,...,-0.021896,2017-01-02,16,2017-01-02,Single Family Residential,NaN,NaN,NaN,NaN,11215747
17,261.0,17,NaN,NaN,NaN,NaN,11229347,NaN,0.0,0.0,...,-0.017167,2017-01-02,17,2017-01-02,Single Family Residential,NaN,NaN,NaN,NaN,11229347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,261.0,77609,2.0,NaN,NaN,1.0,11212539,NaN,3.0,4.0,...,0.020615,2017-09-20,77609,2017-09-20,Single Family Residential,NaN,NaN,NaN,NaN,11212539
77610,261.0,77610,2.0,NaN,NaN,1.0,11212639,NaN,3.0,4.0,...,0.013209,2017-09-21,77610,2017-09-21,Single Family Residential,NaN,NaN,NaN,NaN,11212639
77611,261.0,77611,2.0,NaN,NaN,1.0,11212962,NaN,2.0,3.0,...,0.037129,2017-09-21,77611,2017-09-21,Single Family Residential,NaN,NaN,NaN,NaN,11212962
77612,266.0,77612,2.0,NaN,NaN,1.0,11213162,NaN,3.0,3.0,...,0.007204,2017-09-25,77612,2017-09-25,Condominium,NaN,NaN,NaN,NaN,11213162


### 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

- The input:
    - A dataframe
    - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
    - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
    - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
    - Look up the dropna documentation.
    - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
    - Make use of inplace, i.e. inplace=True/False.


In [35]:
zillow_df.shape

(70252, 74)

In [36]:
prop_req_col = .75
prop_req_row = .5

In [37]:
threshold = int(prop_req_col * len(zillow_df.index))
threshold = int(prop_req_row * len(zillow_df.columns)) 
zillow_df.dropna(axis = 0, thresh = threshold, inplace = True)

In [38]:
zillow_df.shape

(59380, 74)

In [39]:
def drop_nulls(df, prop_req_col = .5 , prop_req_row = .5, inplace = True):
    threshold = int(prop_req_col * len(df.index)) 
    df.dropna(axis = 1, thresh = threshold, inplace = True)
    threshold = int(prop_req_row * len(df.columns)) 
    df.dropna(axis = 0, thresh = threshold, inplace = True)
    return df

In [40]:
drop_nulls(zillow_df, prop_req_col = .75)

,propertylandusetypeid,id,heatingorsystemtypeid,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,taxamount,censustractandblock,heatingorsystemdesc,parcelid.1,logerror,transactiondate,id.1,last_trans_date,propertylandusedesc,parcelid.2
20,261.0,20,2.0,11324547,2.0,4.0,2.0,3633.0,3633.0,6037.0,...,6941.39,NaN,Central,12325145,0.042463,2017-01-02,20,2017-01-02,Single Family Residential,11324547
30,266.0,30,2.0,11524947,2.0,2.0,2.0,1090.0,1090.0,6037.0,...,6165.36,NaN,Central,12713253,0.021853,2017-01-02,30,2017-01-02,Condominium,11524947
31,261.0,31,2.0,11544747,4.0,3.0,4.0,1620.0,1620.0,6037.0,...,10244.94,NaN,Central,14338242,-0.019546,2017-01-02,31,2017-01-02,Single Family Residential,11544747
33,261.0,33,2.0,11585547,2.0,3.0,2.0,2077.0,2077.0,6037.0,...,7924.68,NaN,Central,12155445,-0.040807,2017-01-02,33,2017-01-02,Single Family Residential,11585547
37,266.0,37,2.0,11617547,3.0,2.0,3.0,2735.0,2735.0,6037.0,...,34982.17,NaN,Central,11682477,0.026208,2017-01-02,37,2017-01-02,Condominium,11617547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,261.0,77609,2.0,11212539,3.0,4.0,3.0,1921.0,1921.0,6037.0,...,2860.33,6.037911e+13,Central,11000655,0.020615,2017-09-20,77609,2017-09-20,Single Family Residential,11212539
77610,261.0,77610,2.0,11212639,3.0,4.0,3.0,1780.0,1780.0,6037.0,...,2394.26,6.037911e+13,Central,17239384,0.013209,2017-09-21,77610,2017-09-21,Single Family Residential,11212639
77611,261.0,77611,2.0,11212962,2.0,3.0,2.0,1549.0,1549.0,6037.0,...,3331.81,6.037911e+13,Central,12773139,0.037129,2017-09-21,77611,2017-09-21,Single Family Residential,11212962
77612,266.0,77612,2.0,11213162,3.0,3.0,3.0,1339.0,1339.0,6037.0,...,2510.53,6.037911e+13,Central,12826780,0.007204,2017-09-25,77612,2017-09-25,Condominium,11213162


In [41]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:

        q1, q3 = df[f'{col}'].quantile([.25, .75])  # get quartiles
        
        iqr = q3 - q1   # calculate interquartile range
        
        upper_bound = q3 + k * iqr   # get upper bound
        lower_bound = q1 - k * iqr   # get lower bound

        # return dataframe without outliers
        
        df = df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]
        
    return df

In [42]:
remove_outliers(zillow_df, 1.5, ['bedroomcnt','bathroomcnt','calculatedfinishedsquarefeet'])

,propertylandusetypeid,id,heatingorsystemtypeid,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,...,taxamount,censustractandblock,heatingorsystemdesc,parcelid.1,logerror,transactiondate,id.1,last_trans_date,propertylandusedesc,parcelid.2
30,266.0,30,2.0,11524947,2.0,2.0,2.0,1090.0,1090.0,6037.0,...,6165.36,NaN,Central,12713253,0.021853,2017-01-02,30,2017-01-02,Condominium,11524947
31,261.0,31,2.0,11544747,4.0,3.0,4.0,1620.0,1620.0,6037.0,...,10244.94,NaN,Central,14338242,-0.019546,2017-01-02,31,2017-01-02,Single Family Residential,11544747
33,261.0,33,2.0,11585547,2.0,3.0,2.0,2077.0,2077.0,6037.0,...,7924.68,NaN,Central,12155445,-0.040807,2017-01-02,33,2017-01-02,Single Family Residential,11585547
37,266.0,37,2.0,11617547,3.0,2.0,3.0,2735.0,2735.0,6037.0,...,34982.17,NaN,Central,11682477,0.026208,2017-01-02,37,2017-01-02,Condominium,11617547
72,266.0,72,2.0,12096947,3.0,2.0,3.0,2510.0,2510.0,6037.0,...,9378.06,NaN,Central,11023776,0.093134,2017-01-02,72,2017-01-02,Condominium,12096947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,261.0,77609,2.0,11212539,3.0,4.0,3.0,1921.0,1921.0,6037.0,...,2860.33,6.037911e+13,Central,11000655,0.020615,2017-09-20,77609,2017-09-20,Single Family Residential,11212539
77610,261.0,77610,2.0,11212639,3.0,4.0,3.0,1780.0,1780.0,6037.0,...,2394.26,6.037911e+13,Central,17239384,0.013209,2017-09-21,77610,2017-09-21,Single Family Residential,11212639
77611,261.0,77611,2.0,11212962,2.0,3.0,2.0,1549.0,1549.0,6037.0,...,3331.81,6.037911e+13,Central,12773139,0.037129,2017-09-21,77611,2017-09-21,Single Family Residential,11212962
77612,266.0,77612,2.0,11213162,3.0,3.0,3.0,1339.0,1339.0,6037.0,...,2510.53,6.037911e+13,Central,12826780,0.007204,2017-09-25,77612,2017-09-25,Condominium,11213162


In [43]:
zillow_df = zillow_df.drop(columns=['finishedsquarefeet12','lotsizesquarefeet','regionidcity','regionidzip','censustractandblock'])

In [44]:
zillow_df['latitude'] /= 1000000.
zillow_df['longitude'] /= 1000000.

In [45]:
zillow_df.dropna()

,propertylandusetypeid,id,heatingorsystemtypeid,parcelid,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,fips,fullbathcnt,...,landtaxvaluedollarcnt,taxamount,heatingorsystemdesc,parcelid.1,logerror,transactiondate,id.1,last_trans_date,propertylandusedesc,parcelid.2
20,261.0,20,2.0,11324547,2.0,4.0,2.0,3633.0,6037.0,2.0,...,74104.0,6941.39,Central,12325145,0.042463,2017-01-02,20,2017-01-02,Single Family Residential,11324547
30,266.0,30,2.0,11524947,2.0,2.0,2.0,1090.0,6037.0,2.0,...,124671.0,6165.36,Central,12713253,0.021853,2017-01-02,30,2017-01-02,Condominium,11524947
31,261.0,31,2.0,11544747,4.0,3.0,4.0,1620.0,6037.0,4.0,...,508238.0,10244.94,Central,14338242,-0.019546,2017-01-02,31,2017-01-02,Single Family Residential,11544747
33,261.0,33,2.0,11585547,2.0,3.0,2.0,2077.0,6037.0,2.0,...,436568.0,7924.68,Central,12155445,-0.040807,2017-01-02,33,2017-01-02,Single Family Residential,11585547
37,266.0,37,2.0,11617547,3.0,2.0,3.0,2735.0,6037.0,3.0,...,2164522.0,34982.17,Central,11682477,0.026208,2017-01-02,37,2017-01-02,Condominium,11617547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77609,261.0,77609,2.0,11212539,3.0,4.0,3.0,1921.0,6037.0,3.0,...,32453.0,2860.33,Central,11000655,0.020615,2017-09-20,77609,2017-09-20,Single Family Residential,11212539
77610,261.0,77610,2.0,11212639,3.0,4.0,3.0,1780.0,6037.0,3.0,...,25179.0,2394.26,Central,17239384,0.013209,2017-09-21,77610,2017-09-21,Single Family Residential,11212639
77611,261.0,77611,2.0,11212962,2.0,3.0,2.0,1549.0,6037.0,2.0,...,49747.0,3331.81,Central,12773139,0.037129,2017-09-21,77611,2017-09-21,Single Family Residential,11212962
77612,266.0,77612,2.0,11213162,3.0,3.0,3.0,1339.0,6037.0,3.0,...,29700.0,2510.53,Central,12826780,0.007204,2017-09-25,77612,2017-09-25,Condominium,11213162
